# Settings

# 네이버 웹검색 API

In [ ]:
# 변수 설명:
# client_id: 네이버 API 클라이언트 ID
# client_secret: 네이버 API 클라이언트 시크릿 키
# query: 검색할 키워드 (사용자 입력 값을 URL 인코딩하여 저장)
# idx: 데이터프레임에 저장할 데이터의 인덱스를 추적하는 변수
# display: 한 번의 요청에 가져올 검색 결과의 개수 (최대 100)
# start: 검색 결과의 시작 인덱스
# end: 검색할 페이지의 마지막 인덱스 (총 몇 개의 검색 결과를 가져올지)
# web_df: 검색 결과를 저장할 Pandas DataFrame (열: "Title", "Link", "Description")

import os
import sys
import urllib.request
import pandas as pd
import json
import re

client_id = "S00Vm493qDs_B4TUvI6l"
client_secret = "DXmgZkrRmT"

keyword = input("검색어: ")
query = urllib.parse.quote(keyword)
idx = 0
display = 10
start = 1
end = 50

web_df = pd.DataFrame(columns=("Title", "Link", "Description"))

for start_index in range(start, end, display):
    url = "https://openapi.naver.com/v1/search/webkr?query=" + query + "&display=" + str(display) + "&start=" + str(start_index)
    
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id", client_id)
    request.add_header("X-Naver-Client-Secret", client_secret)
    
    response = urllib.request.urlopen(request)
    rescode = response.getcode()

    if rescode == 200:
        response_body = response.read()
        response_dict = json.loads(response_body.decode('utf-8'))
        items = response_dict['items']
        
        for item_index in range(0, len(items)):
            remove_tag = re.compile('<.*?>')
            title = re.sub(remove_tag, '', items[item_index]['title'])
            link = items[item_index]['link']
            description = re.sub(remove_tag, '', items[item_index]['description'])
            web_df.loc[idx] = [title, link, description]
            idx += 1
    else:
        print("Error Code:" + str(rescode))

# web_df 데이터프레임 출력
web_df


### 웹 검색 결과 저장

In [13]:
# DataFrame을 CSV 파일로 저장
web_df.to_csv(f"web_df_{keyword}.csv", index=False)

### 결과 요약

In [ ]:
# 변수 설명:
# client_id: 네이버 API 클라이언트 ID
# client_secret: 네이버 API 클라이언트 시크릿 키
# query: 검색할 키워드 (사용자 입력 값을 URL 인코딩하여 저장)
# idx: 데이터프레임에 저장할 데이터의 인덱스를 추적하는 변수
# display: 한 번의 요청에 가져올 검색 결과의 개수 (최대 100)
# start: 검색 결과의 시작 인덱스
# end: 검색할 페이지의 마지막 인덱스 (총 몇 개의 검색 결과를 가져올지)
# web_df: 검색 결과를 저장할 Pandas DataFrame (열: "Title", "Link", "Description")

import os
import openai
import sys
import urllib.request
import pandas as pd
import json
import re
from dotenv import load_dotenv 
# API KEY 정보를 불러옵니다
load_dotenv('./credentials/.env')

api_key = os.environ.get("OPENAI_API_KEY")
client_id = os.environ.get("CLIENT_ID")
client_secret = os.environ.get("CLIENT_SECRET")

# OpenAI GPT 요약 함수 정의 (한국어)
def summarize_text(text):
    openai.api_key = api_key
    completion = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "당신은 텍스트를 요약해주는 유용한 조수입니다."},
            {"role": "user", "content": f"다음 텍스트를 요약해 주세요:\n\n{text}"}
        ]
    )
    return completion.choices[0].message.content

keyword = input("검색어: ")
query = urllib.parse.quote(keyword)
idx = 0
display = 10
start = 1
end = 50

web_df = pd.DataFrame(columns=("Title", "Link", "Description", "Summary"))

for start_index in range(start, end, display):
    url = "https://openapi.naver.com/v1/search/webkr?query=" + query + "&display=" + str(display) + "&start=" + str(start_index)

    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id", client_id)
    request.add_header("X-Naver-Client-Secret", client_secret)

    response = urllib.request.urlopen(request)
    rescode = response.getcode()

    if rescode == 200:
        response_body = response.read()
        response_dict = json.loads(response_body.decode('utf-8'))
        items = response_dict['items']

        for item_index in range(0, len(items)):
            remove_tag = re.compile('<.*?>')
            title = re.sub(remove_tag, '', items[item_index]['title'])
            link = items[item_index]['link']
            description = re.sub(remove_tag, '', items[item_index]['description'])
            # 요약 추가 
            summary = summarize_text(description)
            
            web_df.loc[idx] = [title, link, description, summary]
            idx += 1
    else:
        print("Error Code:" + str(rescode))

# web_df 데이터프레임 출력
web_df

In [20]:
# DataFrame을 CSV 파일로 저장
web_df.to_csv(f"web_df_summary_{keyword}.csv", index=False)

### 상위 10개의 키워드 추출 (예정)

### 키워드에 대한 긍정/부정 반응에 대한 워드클라우드 (예정)

# 네이버 뉴스 검색 API

In [ ]:
# 변수 설명:
# client_id: 네이버 API 클라이언트 ID
# client_secret: 네이버 API 클라이언트 시크릿 키
# query: 검색할 키워드 (사용자 입력 값을 URL 인코딩하여 저장)
# idx: 데이터프레임에 저장할 데이터의 인덱스를 추적하는 변수
# display: 한 번의 요청에 가져올 검색 결과의 개수 (최대 100)
# start: 검색 결과의 시작 인덱스
# end: 검색할 페이지의 마지막 인덱스 (총 몇 개의 검색 결과를 가져올지)
# sort: 검색 결과를 정렬하는 기준 (sim: 유사도, date: 날짜 순)
# news_df: 검색 결과를 저장할 Pandas DataFrame (열: "Title", "Original Link", "Link", "Description", "Publication Date")

import os
import sys
import urllib.request
import pandas as pd
import json
import re

client_id = "S00Vm493qDs_B4TUvI6l"
client_secret = "DXmgZkrRmT"

keyword = input("검색어: ")
query = urllib.parse.quote(keyword)
idx = 0
display = 100
start = 1
end = 1000
sort = "sim"

news_df = pd.DataFrame(columns=("Title", "Original Link", "Link", "Description", "Publication Date"))

for start_index in range(start, end, display):
    url = "https://openapi.naver.com/v1/search/news?query=" + query + "&display=" + str(display) + "&start=" + str(start_index) + "&sort=" + sort
    
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id", client_id)
    request.add_header("X-Naver-Client-Secret", client_secret)
    
    response = urllib.request.urlopen(request)
    rescode = response.getcode()

    if rescode == 200:
        response_body = response.read()
        response_dict = json.loads(response_body.decode('utf-8'))
        items = response_dict['items']
        
        for item_index in range(0, len(items)):
            remove_tag = re.compile('<.*?>')
            title = re.sub(remove_tag, '', items[item_index]['title'])
            original_link = items[item_index]['originallink']
            link = items[item_index]['link']
            description = re.sub(remove_tag, '', items[item_index]['description'])
            pub_date = re.sub(remove_tag, '', items[item_index]['pubDate'])
            
            news_df.loc[idx] = [title, original_link, link, description, pub_date]
            idx += 1
    else:
        print("Error Code:" + str(rescode))

# news_df 데이터프레임 출력
news_df

### 뉴스 검색 결과 저장

In [16]:
# DataFrame을 CSV 파일로 저장
news_df.to_csv(f"news_df_{keyword}.csv", index=False)

### 뉴스 결과 요약

In [21]:
import os
import openai
import sys
import urllib.request
import pandas as pd
import json
import re
from dotenv import load_dotenv 
# API KEY 정보를 불러옵니다
load_dotenv('./credentials/.env')

api_key = os.environ.get("OPENAI_API_KEY")
client_id = os.environ.get("CLIENT_ID")
client_secret = os.environ.get("CLIENT_SECRET")

# OpenAI GPT 요약 함수 정의 (100자 이내)
def summarize_text(text):
    openai.api_key = api_key
    completion = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "당신은 텍스트를 한 문단으로 요약해주는 유용한 조수입니다."},
            {"role": "user", "content": f"다음 텍스트를 199자 이내로 한 문단으로 요약해 주세요:\n\n{text}"}
        ]
    )
    return completion.choices[0].message.content


keyword = input("검색어: ")
query = urllib.parse.quote(keyword)
idx = 0
display = 100
start = 1
end = 1000
sort = "sim"

news_df = pd.DataFrame(columns=("Title", "Original Link", "Link", "Description", "Publication Date", "Summary"))

for start_index in range(start, end, display):
    url = "https://openapi.naver.com/v1/search/news?query=" + query + "&display=" + str(display) + "&start=" + str(start_index) + "&sort=" + sort

    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id", client_id)
    request.add_header("X-Naver-Client-Secret", client_secret)

    response = urllib.request.urlopen(request)
    rescode = response.getcode()

    if rescode == 200:
        response_body = response.read()
        response_dict = json.loads(response_body.decode('utf-8'))
        items = response_dict['items']

        for item_index in range(0, len(items)):
            remove_tag = re.compile('<.*?>')
            title = re.sub(remove_tag, '', items[item_index]['title'])
            original_link = items[item_index]['originallink']
            link = items[item_index]['link']
            description = re.sub(remove_tag, '', items[item_index]['description'])
            pub_date = re.sub(remove_tag, '', items[item_index]['pubDate'])
            
            # 요약 추가 (100자 이내 한 문단)
            summary = summarize_text(description)
            
            news_df.loc[idx] = [title, original_link, link, description, pub_date, summary]
            idx += 1
    else:
        print("Error Code:" + str(rescode))

# news_df 데이터프레임 출력
news_df


In [ ]:
# DataFrame을 CSV 파일로 저장
news_df.to_csv(f"news_df_summary_{keyword}.csv", index=False)

### 뉴스의 일별 언급량에 따른 주식 가격 예측 (예정)